# Getting access to FHIR resources

Once you have a `service` object, you can read and search all the resources advertised by the FHIR server.

Normally, you'll create an authenticated version of a `service` using an OAuth2 token:

```python
from fhirstorm import Connection
from requests_oauthlib import OAuth2Session
conn = Connection(
    SMART_SERVICE_ROOT, 
    session=OAuth2Session(token=ACCESS_TOKEN))
svc = conn.service()
```

For this tutorial, we'll just use the public [SmartHealthIT][smarthealthit] sandboxes, and so we won't need to use authentication.
The only differences are that we'll use the **open** endpoint and we won't worry about the session:

[smarthealthit]: http://docs.smarthealthit.org/

In [1]:
from fhirstorm import Connection
SMART_SERVICE_ROOT = 'https://sb-fhir-dstu2.smarthealthit.org/smartdstu2/open'
conn = Connection(SMART_SERVICE_ROOT)
service = conn.service()

# Getting the patient data

Generally, the patient ID is sent back to us with the token. 
Sometimes, however, we might have to decode the access or id token to get it.
That's covered in the authorization tutorial. 

For now, let's just assume that we know we're dealing with patient `'783a1f02-5bf9-41c9-90d0-c2c4d35d3ec3'`

In [2]:
patient_id = '783a1f02-5bf9-41c9-90d0-c2c4d35d3ec3'

# FHIR Resources

All the resources supported by the FHIR server are exposed as attributes of the `service.r` object.

The first one we might want to get is the `Patient`:

In [3]:
service.r.Patient.fetch(patient_id)

<Bound Patient>

In [4]:
service.r.Patient.metadata

{'bind': <fhirstorm.util.AttrProxy at 0x111f437b8>,
 'conditionalCreate': True,
 'conditionalDelete': 'multiple',
 'conditionalUpdate': True,
 'extension': [{'url': 'http://hl7api.sourceforge.net/hapi-fhir/res/extdefs.html#resourceCount', 'valueDecimal': 1560}],
 'interaction': [{'code': 'read'},
  {'code': 'vread'},
  {'code': 'update'},
  {'code': 'delete'},
  {'code': 'history-instance'},
  {'code': 'history-type'},
  {'code': 'create'},
  {'code': 'search-type'}],
 'profile': {'reference': 'http://hl7.org/fhir/profiles/Patient'},
 'searchInclude': ['*',
  'Patient:careprovider',
  'Patient:link',
  'Patient:organization'],
 'searchParam': [{'name': '_content', 'type': 'string', 'documentation': "Search the contents of the resource's data using a fulltext search"},
  {'name': '_has', 'type': 'string', 'documentation': 'Return resources linked to by the given target'},
  {'name': '_id', 'type': 'string', 'documentation': 'The ID of the resource'},
  {'name': '_language', 'type': 'str

# Getting a list of all the types of resources available

Since the `service.r` object has a list of all the resources, you can use it to find out what types
of resources the FHIR server supports:

In [5]:
dir(service.r)

['Account',
 'AllergyIntolerance',
 'Appointment',
 'AppointmentResponse',
 'AuditEvent',
 'Basic',
 'Binary',
 'BodySite',
 'Bundle',
 'CarePlan',
 'Claim',
 'ClaimResponse',
 'ClinicalImpression',
 'Communication',
 'CommunicationRequest',
 'Composition',
 'ConceptMap',
 'Condition',
 'Conformance',
 'Contract',
 'Coverage',
 'DataElement',
 'DetectedIssue',
 'Device',
 'DeviceComponent',
 'DeviceMetric',
 'DeviceUseRequest',
 'DeviceUseStatement',
 'DiagnosticOrder',
 'DiagnosticReport',
 'DocumentManifest',
 'DocumentReference',
 'EligibilityRequest',
 'EligibilityResponse',
 'Encounter',
 'EnrollmentRequest',
 'EnrollmentResponse',
 'EpisodeOfCare',
 'ExplanationOfBenefit',
 'FamilyMemberHistory',
 'Flag',
 'Goal',
 'Group',
 'HealthcareService',
 'ImagingObjectSelection',
 'ImagingStudy',
 'Immunization',
 'ImmunizationRecommendation',
 'ImplementationGuide',
 'List',
 'Location',
 'Media',
 'Medication',
 'MedicationAdministration',
 'MedicationDispense',
 'MedicationOrder',
 'M

In [6]:
service.r.Account.metadata

{'bind': <fhirstorm.util.AttrProxy at 0x11046b8d0>,
 'conditionalCreate': True,
 'conditionalDelete': 'multiple',
 'conditionalUpdate': True,
 'interaction': [{'code': 'read'},
  {'code': 'vread'},
  {'code': 'update'},
  {'code': 'delete'},
  {'code': 'history-instance'},
  {'code': 'history-type'},
  {'code': 'create'},
  {'code': 'search-type'}],
 'profile': {'reference': 'http://hl7.org/fhir/profiles/Account'},
 'searchInclude': ['*', 'Account:owner', 'Account:patient', 'Account:subject'],
 'searchParam': [{'name': '_content', 'type': 'string', 'documentation': "Search the contents of the resource's data using a fulltext search"},
  {'name': '_has', 'type': 'string', 'documentation': 'Return resources linked to by the given target'},
  {'name': '_id', 'type': 'string', 'documentation': 'The ID of the resource'},
  {'name': '_language', 'type': 'string', 'documentation': 'The language of the resource'},
  {'name': '_lastUpdated', 'type': 'date', 'documentation': 'Only return resourc

Most, but not all, resources support searching, so we can see what this particular patient record has:

In [7]:
import requests
for resourceType, res in service.r.items():
    params = {sp.name: sp for sp in res.metadata.get('searchParam', [])}
    print(resourceType)
    if 'patient' not in params: 
        print ('- (not searchable by patient)')
        print()
        continue
    try:
        bundle = res.search(dict(patient=patient_id))
        print(f'- {bundle.total} {resourceType}s for this patient')
    except requests.HTTPError as err:
        print(f'- [ERROR]: {type(err)}')
        print(f'         : {err.response.headers["content-type"]}')
        if err.response.headers['content-type'].startswith('application/json'):
            print(f'         : {err.response.json().get("message")}')
        else:
            print(f'         : {err.response.content}')
    print()

Account
- 0 Accounts for this patient

AllergyIntolerance
- 0 AllergyIntolerances for this patient

Appointment
- 0 Appointments for this patient

AppointmentResponse
- 0 AppointmentResponses for this patient

AuditEvent
- 0 AuditEvents for this patient

Basic
- 0 Basics for this patient

Binary
- (not searchable by patient)

BodySite
- 0 BodySites for this patient

Bundle
- (not searchable by patient)

CarePlan
- 0 CarePlans for this patient

Claim
- 0 Claims for this patient

ClaimResponse
- (not searchable by patient)

ClinicalImpression
- 0 ClinicalImpressions for this patient

Communication
- 0 Communications for this patient

CommunicationRequest
- 0 CommunicationRequests for this patient

Composition
- 0 Compositions for this patient

ConceptMap
- (not searchable by patient)

Condition
- 5 Conditions for this patient

Conformance
- (not searchable by patient)

Contract
- 0 Contracts for this patient

Coverage
- (not searchable by patient)

DataElement
- (not searchable by patien